# BINF3020 Lab 1 – Expression data analysis in Python: Questions

*Last revision:Wed 29 Oct 2025 22:31:42 AEDT.*  Revised version.

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container {width:90% !important;}</style>"))

## Introduction

In this lab/homework activity, you will answer some questions on topics from lectures and learn how to apply some of the techniques covered to representative datasets. We will be working with a number of microarray datasets and working on data preparation, cluster analysis and classification learning. You need to work through the notebook and complete the answers to questions in the notebook cells.

#### Please ensure each answer is completed in the single cell of the notebook immediately following the question, and that all answers are in ```markdown``` format !!!

Once you have completed the notebook, make sure you save it with the filename "Lab1_Solutions.ipynb" and submit it via Moodle.

All questions are worth 1 mark, so there are a total of *10 marks* available.

Total notebook marks will be scaled to a **course mark out of 5** to contribute to your course total.

**Deadline: 23:55pm, Monday November 10, 2025**.

Your environment should have already been set up to enable to run this notebook if you successfully completed the lab "Lab1_Intro_Python.ipynb". The main data structure we will be using is the ```DataFrame``` (something you are probably familiar with from R). The principal toolkit for handling DataFrames in Python is a library called ```Pandas```. You can find some useful information in compact form on basic use of [Pandas](https://towardsdatascience.com/how-to-master-pandas-8514f33f00f6) and another one on some more [advanced uses](https://towardsdatascience.com/learn-advanced-features-for-pythons-main-data-analysis-library-in-20-minutes-d0eedd90d086).

## Data Exploration

As a first step, whenever we get some data given to us we will start by doing some basic exploration ("know your data"). This will give us an idea of what is in the data and what types of analysis we may be able to do. 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

df_train = pd.read_csv("./datasets/ALL_AML_train.csv")
df_train.shape

As we can see above, there are over 7,000 rows and 78 columns. From our knowledge of microarray data we would assume that the shape of this data frame suggests the rows correspond to genes and the columns to samples. Let us transpose the dataset and look at the first 5 rows.

In [ ]:
df_train.head(5).T

This appears to be Affymetrix data. 
However, it also looks like this dataset needs to be "cleaned" before starting any analysis, so that is what we will do next.

In [ ]:
# pd.set_option('future.no_silent_downcasting', True)

# Read in the training and test datasets
train_dataset = pd.read_csv('./datasets/ALL_AML_train.csv')
test_dataset  = pd.read_csv('./datasets/ALL_AML_test.csv')

train_dataset1 = [col for col in train_dataset.columns if "call" not in col]
train_dataset  = train_dataset[train_dataset1]

test_dataset1 = [col for col in test_dataset.columns if "call" not in col]
test_dataset  = test_dataset[test_dataset1]

train_dataset = train_dataset.T
test_dataset = test_dataset.T

train_dataset2 = train_dataset.drop(['Gene Description','Gene Accession Number'],axis=0)
test_dataset2  = test_dataset.drop(['Gene Description','Gene Accession Number'],axis=0)

# Ensure data frame entries are numeric and indexing has the same order
train_dataset2.index = pd.to_numeric(train_dataset2.index)
train_dataset2.sort_index(inplace=True)

test_dataset2.index = pd.to_numeric(test_dataset2.index)
test_dataset2.sort_index(inplace=True)

print(f'Dataset sizes: ')
print(f'Training set: {train_dataset2.shape}')
print(f'Test set: {test_dataset2.shape}')

# Read in the classes and inspect the structure of the data
y = pd.read_csv('./datasets/ALL_AML_classes.csv')
print(f'Class distribution: ')
print(y['cancer'].value_counts())
y = y.replace({'ALL':0,'AML':1})

We need to do some pre-processing to make sure the data is in a suitable format for the sklearn algorithms we will use, and also to scale the gene expression values as a normalization step.

In [ ]:
# Training set
X_train = train_dataset2.reset_index(drop=True)
Y_train = y[y.patient <= 38].reset_index(drop=True)

# Test set
X_test = test_dataset2.reset_index(drop=True)
Y_test = y[y.patient > 38].reset_index(drop=True)

Y_test = Y_test.iloc[:,1].values
Y_train = Y_train.iloc[:,1].values

# refer to the Scikit-learn documentation to understand how StandardScaler works
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

Since the data is very high-dimensional and we only have a relatively small number of samples we will apply _dimensionality reduction_ using PCA.

**Q1. What does it mean to say that a dataset "is very high-dimensional" ?**

1. The dataset has a small number of samples and a small number of features
2. The dataset has a small number of samples and a large number of features
3. The dataset has a large number of samples and a small number of features
4. The dataset has a large number of samples and a large number of features
5. I have a different interpretation (write a sentence below explaining why you selected this answer)

**Q1 answer.**   _Your answer goes here._ 

Now we turn to address the issue of high-dimensionality with PCA.

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components = None)
X_train_pca = pca.fit_transform(X_train)


In [ ]:
# Calculating the "explained variance" of each principal component, up to 90% of the total variance
total = sum(pca.explained_variance_)
k = 0
current_variance = 0
while current_variance/total < 0.90:
      current_variance += pca.explained_variance_[k]
      k = k + 1
k

# Applying PCA to select the top k components capturing up to 90% of the variance
pca = PCA(n_components = k)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

var_exp = pca.explained_variance_ratio_.cumsum()
var_exp = var_exp*100

var1=np.cumsum(np.round(pca.explained_variance_ratio_, decimals=4)*100)
print(f'PCA Dim: {k}; PCA Var: {var1}')

In [ ]:
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance')
plt.plot(np.cumsum(pca.explained_variance_ratio_))

**Q2. what is the reduction in the number of dimensions if we use the PCA transformed data instead of the original ?**

1. 28
2. 38
3. 72
4. 7101
5. 7129

**Q2 answer.**   _Your answer goes here._ 

**Q3. In your own words, briefly explain what inference can be drawn from the results of this application of PCA to the dataset.**

**Q3 answer.**   _Your answer goes here._

Now to visualize the data following this transform, we select only the first two principal components.

In [ ]:
import matplotlib.cm

# plt.scatter(X_train_pca[:, 0], X_train_pca[:, 1],c=Y_train, edgecolor='none', alpha=0.9, cmap=plt.cm.get_cmap('Spectral', 10))

# 'Qualitative' 'Setl'
plt.scatter(X_train_pca[:, 0], X_train_pca[:, 1],c=Y_train, edgecolor='none', alpha=0.9, cmap=matplotlib.colormaps['Accent'])

plt.xlabel('component 1')
plt.ylabel('component 2')
plt.title('Separation of classes by first two principal components')
plt.colorbar();

**Q4.  In your opinion, which of the following is the best explanation of the plot of the separation of the two classes when the data are projected only onto the first two principal components ?**

1. There is good class separation due to the large amount of explained variance from the first two principal components
2. There is only limited class separation although there is a large amount of explained variance from the first two principal components
3. There is good class separation although there is only a limited amount of explained variance from the first two principal components
4. There is only limited class separation due to the limited amount of explained variance from the first two principal components
5. None of the above

**Q4 answer.**   _Your answer goes here._ 

Alternatively, we could apply one of the methods designed to reduce dimensionality "automatically" to a pre-defined number of dimensions (Self-Organizing Maps, discussed in lectures, is one such method). Methods designed to enable visualisation, map input data to two dimensions. One such method is [t-SNE](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html), which we will use to provde another perspective on the data. You can find more information on t-SNE [here](https://lvdmaaten.github.io/tsne/).

In [ ]:
from sklearn import manifold

X = X_train
tsne = manifold.TSNE(n_components=2, init="random", random_state=0, perplexity=3)
res = tsne.fit_transform(X)

fig, ax = plt.subplots()
ax.scatter(res[:, 0], res[:, 1], c=Y_train, edgecolor='none', alpha=0.9, cmap=matplotlib.colormaps['Accent'])
plt.show()

**Q5. Do you think the results from applying tSNE to the original data are consistent with your answer to Q4 ? Answer yes or no, and provide a one-sentence justification for your choice.**

**Q5 answer.**  _Your answer goes here._ 

## Classification

We have two options here. On one hand, now we know that dimensionality can be reduced, for example by PCA, we can try a standard linear classifier. However, since the data will be transformed by such dimensionality reduction it is not clear how this new representation relates to the original gene expression dataset. On the other hand, we could try a more powerful "black-box" classifier, such as ensemble learning using boosting on the original data, and see how that turns out.

To evaluate and compare two classifiers we will apply several widely-used metrics. First is the predictive accuracy of each classifer on the test set. Second, the 2x2 [confusion matrices](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html) for the test set predictions are generated. Third, the [Mathews correlation coefficient (MCC)](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.matthews_corrcoef.html) is computed from the confusion matrices for both classifiers.

If you are not familiar with confusion matrices, here is the format of the confusion matrix evaluating a two-class predictive classifier, where the classes are denoted "positive" (Pos) and "negative" (Neg), although these are simply arbitrary labels. You can find more information on confusion matrices [here](https://en.wikipedia.org/wiki/Confusion_matrix).

| Actual | Predicted |   |
| :---   | :---: | :---: |
|        | Pos   | Neg   |
| Pos    | TP    | FN    |
| Neg    | FP    | TN    |


The Mathews correlation coefficient is often used in bioinformatics applications of machine learning as a single metric summarising the results in a confusion matrix. Useful information is in the Introduction of this [article](https://www.frontiersin.org/articles/10.3389/frobt.2022.876814/full) and more detail is in this [Wikipedia article](https://en.wikipedia.org/wiki/Phi_coefficient).

In [ ]:
# Metrics for evaluation
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import matthews_corrcoef

In [ ]:
# Fitting a logistic regression classifier to the PCA-transformed version of the training set
from sklearn.linear_model import LogisticRegression

clf=LogisticRegression(penalty='l2',solver='liblinear',random_state=0)
clf.fit(X_train_pca,Y_train)

Y_pred=clf.predict(X_test_pca)

log_reg_ac = accuracy_score(Y_test, Y_pred)
log_reg_cm = confusion_matrix(Y_test, Y_pred)

print(f'Logistic Regression Accuracy: {log_reg_ac}')
print(log_reg_cm)

matthews_corrcoef(Y_test, Y_pred)

In [ ]:
# Fitting a gradient-boosting classifier to the training set
from sklearn.ensemble import GradientBoostingClassifier

clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
clf.fit(X_train, Y_train)
Y_pred = clf.predict(X_test)

grad_boost_ac = accuracy_score(Y_test, Y_pred)
grad_boost_cm = confusion_matrix(Y_test, Y_pred)

print(f'Gradient Boosting Accuracy: {grad_boost_ac}')
print(grad_boost_cm)

matthews_corrcoef(Y_test, Y_pred)

**Q6 [1 mark]. From these results, what can you say about which classifier may be "better"?**

**Q6 answer.**   _Your answer goes here. [HINT: compare both accuracies]_  



**Q7 [1 mark]. With reference to the materials above on confusion matrices, etc. provide some commentary on any further insight you might obtain about the relative performance of the two classification methods.**

**Q7 answer.** _Your answer goes here. [HINT: think about what may be going on beyond accuracy]_



## Clustering

Turning our attention back to unsupervised learning, we start by loading the Yeast cell-cycle time series dataset.

In [ ]:
# Load the Spellman dataset
df_sc = pd.read_csv("./datasets/gene800_Spellman.csv")
df_sc.shape
df_sc.info()
df_sc.head(2).T

In [ ]:
train_sc = df_sc.T
train_sc2 = train_sc.drop(['gene_name'], axis=0)
train_sc2.head(3)

Before clustering, try a visualization with t_SNE.

In [ ]:
tsne_sc = manifold.TSNE(n_components=2, init="random", random_state=0, perplexity=10)
res_sc = tsne_sc.fit_transform(train_sc2.T)

fig, ax = plt.subplots()
ax.scatter(res_sc[:, 0], res_sc[:, 1])
plt.show()

**Q8. What can you infer about the possibility of finding a good clustering of this data from the t-SNE visualization ?**

**Q8 answer.**   _Your answer goes here._


In [ ]:
from sklearn.cluster import KMeans

km_sc = KMeans(n_clusters=10, n_init=10)
km_sc.fit(train_sc2)

In [ ]:
from sklearn import metrics

for k in range(2,20):
    km_sc = KMeans(n_clusters=k, n_init=10)
#     km_sc.fit(train_sc2)
    km_sc.fit(train_sc2.T)
    lb_sc = km_sc.labels_
#     ss = metrics.silhouette_score(train_sc2, lb_sc, metric='sqeuclidean')
    ss = metrics.silhouette_score(train_sc2.T, lb_sc, metric='sqeuclidean')
    print(f'K = {k}, Silhouette score = {ss}')

**Q9. Which setting for "k" achieves the best clustering ?**

**Q9 answer.**   _Your answer goes here._ 


In [ ]:
# Load the labels for the NCI60 dataset
df_nci_lb = pd.read_csv("./datasets/nci60_labs.csv")
print(df_nci_lb.shape)
df_nci_lb.x.value_counts(sort=True)

In [ ]:
# Load the microarray data for the NCI60 dataset
df_nci_dat = pd.read_csv("./datasets/nci60_data.csv")
print(df_nci_dat.shape)
print(df_nci_dat.T.head(3))

In [ ]:

df_nci_X = df_nci_dat.iloc[0:64,1:6831]

from sklearn.cluster import AgglomerativeClustering

avg = AgglomerativeClustering(linkage='average', metric='euclidean', n_clusters=20) 
avg.fit(df_nci_X)

nci_lbls = avg.labels_

nci_ss = metrics.silhouette_score(df_nci_X, nci_lbls, metric='sqeuclidean')
print(f'Silhouette score = {nci_ss}')

linkages = ['average', 'single', 'complete', 'ward']
for link_type in linkages:
    avg = AgglomerativeClustering(linkage=link_type, metric='euclidean', n_clusters=10)
    avg.fit(df_nci_X)
    nci_lbls = avg.labels_
    nci_ss = metrics.silhouette_score(df_nci_X, nci_lbls, metric='sqeuclidean')
    print(f'Silhouette score = {nci_ss}')
                                          

**Q10. Can you find a better clustering by changing the number of clusters (set by the 'n_clusters=' parameter ?**

**Q10 answer.** _Add your code in the cell below and run it to generate output for the silhouette scores._
